### Exploring how to setup the Twitter API

- The code is currently really dirty and is basically a copy-past from https://towardsdatascience.com/python-identifying-twitter-influencers-through-network-analysis-964c9b293e03

In [16]:
import sys
import tweepy
import igraph
import networkx as nx
import pandas as pd
import numpy as np
import json
import csv
import ast
from operator import itemgetter
from igraph import *
import re

In [17]:
# These are mine, I guess you should create your own developer account. 
CONSUMER_KEY = 'JbQ88IvxR7pAqtiGN9V2gcEfT'
CONSUMER_SECRET_KEY = 'KtbFDwFfub5QzBmyUxpxdvOMR0kfS4u2i2C4T82jrDSTinP74Q'

ACCESS_TOKEN = '1074946114917404672-em24Gf9a9Ouc5907wJhvmmAn75qeZy'
ACCESS_SECRET_TOKEN = 'nDfuT1tO7Ub2QLie4AasUGkvagS7nHKB9L65gJFjS79iH'

In [33]:
class TweetGrabber():
    
    def __init__(self,myApi,sApi,at,sAt):
        self.tweepy = tweepy
        auth = tweepy.OAuthHandler(myApi, sApi)
        auth.set_access_token(at, sAt)
        self.api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

    #Return the string without non ASCII characters
    def strip_non_ascii(self,string):
        stripped = (c for c in string if 0 < ord(c) < 127)
        return ''.join(stripped)  

    def user_search(self,user,csv_prefix):
        API_results = self.tweepy.Cursor(self.api.user_timeline,id=user,tweet_mode='extended').items()
        with open(f'{csv_prefix}.csv', 'w', newline='') as csvfile:
            fieldnames = ['tweet_id', 'tweet_text', 'date', 'user_id', 'user_mentions', 'retweet_count']
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            writer.writeheader()

            for tweet in API_results:
                text = self.strip_non_ascii(tweet.full_text)
                date = tweet.created_at.strftime('%m/%d/%Y')        
                writer.writerow(
                  {
                    'tweet_id': tweet.id_str,
                    'tweet_text': text,
                    'date': date,
                    'user_id': tweet.user.id_str,
                    'user_mentions':tweet.entities['user_mentions'],
                    'retweet_count': tweet.retweet_count
                    }
                )

# Process the created CSV in order to generate edge list
class RetweetParser():
    
    def __init__(self,data,user):
        self.user = user

        edge_list = []
  
        for idx,row in data.iterrows():
            if len(row[4]) > 5:    
                user_account = user
                weight = np.log(row[5] + 1)
                for idx_1, item in enumerate(ast.literal_eval(row[4])):
                    edge_list.append((user_account,item['screen_name'],weight))

                    for idx_2 in range(idx_1+1,len(ast.literal_eval(row[4]))):
                        name_a = ast.literal_eval(row[4])[idx_1]['screen_name']
                        name_b = ast.literal_eval(row[4])[idx_2]['screen_name']

                        edge_list.append((name_a,name_b,weight))
      
        with open(f'{self.user}.csv', 'w', newline='') as csvfile:
            fieldnames = ['user_a', 'user_b', 'log_retweet']
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            writer.writeheader()

            for row in edge_list:        
                writer.writerow({
                              'user_a': row[0],
                              'user_b': row[1],
                              'log_retweet': row[2]
                              })


# Eigenvector centrality measures 'influence' of each node within the graph network
class TweetGraph():
    def __init__(self,edge_list):
        data = pd.read_csv(edge_list).to_records(index=False)
        self.tuple_graph = igraph.Graph.TupleList(data, weights=True, directed=False)
        
    def e_centrality(self):
        vectors = self.tuple_graph.eigenvector_centrality()
        e = {name:cen for cen, name in  zip([v for v in vectors],self.tuple_graph.vs['name'])}
        return sorted(e.items(), key=operator.itemgetter(1),reverse=True)


In [51]:
t = TweetGrabber(
  myApi=CONSUMER_KEY,
  sApi=CONSUMER_SECRET_KEY,
  at=ACCESS_TOKEN,
  sAt=ACCESS_SECRET_TOKEN
)

# screen_name = 125815552
screen_name = 'GuillaumeJaume'
t.user_search(user=screen_name, csv_prefix=screen_name)

# Read the created CSV into a pandas DataFrame for input to RetweetParser class
userFrame = pd.read_csv(screen_name + ".csv")

# RetweetParser overwrites the first CSV with a weighted edgelist
r = RetweetParser(userFrame, screen_name)

# The weighted, undirected iGraph object
log_graph = TweetGraph(edge_list= screen_name + ".csv")

# Add 'size' attribute to each vertex based on its Eigencentrality
# NOTE: multiplying the value by some consistent large number creates a more intuitive
# plot, viewing-wise, but doesn't impact classification, since this change is applied
# to all vertices
for key, value in log_graph.e_centrality():
    log_graph.tuple_graph.vs.find(name=key)['size'] = value*20

# Save the graph in GML format
print("Building gml...", log_graph)
log_graph.tuple_graph.write_gml(f=screen_name+".gml")

log_graph.tuple_graph.summary()
#Plot the graph for viewing
# style = {}
# style["edge_curved"] = False
# style["vertex_label"] = log_graph.tuple_graph.vs['name']
# style["vertex_label_size"] = 5
# plot(log_graph.tuple_graph, **style)

Building gml... <__main__.TweetGraph object at 0x7fd3f0ac3e10>


'IGRAPH UNW- 21 92 -- \n+ attr: name (v), size (v), weight (e)'